LangServe는 LangChain 앱을 REST API로 배포

https://github.com/teddylee777/langserve_ollama?tab=readme-ov-file

**<server.py>**

In [ ]:
from fastapi import FastAPI
from fastapi.responses import RedirectResponse
from fastapi.middleware.cors import CORSMiddleware
from typing import List, Union
from langserve.pydantic_v1 import BaseModel, Field
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langserve import add_routes
from chain import chain
from chat import chain as chat_chain
from translator import chain as EN_TO_KO_chain
from llm import llm as model
from xionic import chain as xionic_chain

app = FastAPI()

# Set all CORS enabled origins
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["*"],
)

@app.get("/")
async def redirect_root_to_docs():
    return RedirectResponse("/xionic/playground")


add_routes(app, chain, path="/prompt")


class InputChat(BaseModel):
    """Input for the chat endpoint."""

    messages: List[Union[HumanMessage, AIMessage, SystemMessage]] = Field(
        ...,
        description="The chat messages representing the current conversation.",
    )

add_routes(
    app,
    chat_chain.with_types(input_type=InputChat),
    path="/chat",
    enable_feedback_endpoint=True,
    enable_public_trace_link_endpoint=True,
    playground_type="chat",
)

add_routes(app, EN_TO_KO_chain, path="/translate")

add_routes(app, model, path="/llm")

add_routes(
    app,
    xionic_chain.with_types(input_type=InputChat),
    path="/xionic",
    enable_feedback_endpoint=True,
    enable_public_trace_link_endpoint=True,
    playground_type="chat",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)

실행
중요: server.py외에 langserve_ollama-main\app 디렉토리 내 파일 전체가 server.py와 동일한 디렉토리에 있어야 함.

In [ ]:
$ python server.py

In [ ]:
# <xionic.py>
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser


#llm = ChatOpenAI(
    #base_url="http://sionic.chat:8001/v1",
    #api_key="934c4bbc-c384-4bea-af82-1450d7f8128d",
    #model="xionic-ko-llama-3-70b",
#)

llm = ChatOllama(model="Test1-10.8B")

# Prompt 설정
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful, smart, kind, and efficient AI assistant named '테디'. You always fulfill the user's requests to the best of your ability. You must generate an answer in Korean.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm | StrOutputParser()


방법2 : 수동 실행

In [ ]:
OLLAMA_HOST=192.168.35.92:11434 ollama serve OLLAMA_HOST=192.168.35.92:11434 ollama serve

## RAG

**검색 단계(Retrieval Phase)**: 사용자의 질문이나 컨텍스트를 입력으로 받아서, 이와 관련된 외부 데이터를 검색하는 단계입니다. 이 때 검색 엔진이나 데이터베이스 등 다양한 소스에서 필요한 정보를 찾아냅니다. 검색된 데이터는 질문에 대한 답변을 생성하는데 적합하고 상세한 정보를 포함하는 것을 목표로 합니다.

**생성 단계(Generation Phase)**: 검색된 데이터를 기반으로 LLM 모델이 사용자의 질문에 답변을 생성하는 단계입니다. 이 단계에서 모델은 검색된 정보와 기존의 지식을 결합하여, 주어진 질문에 대한 답변을 생성합니다.

![image.png](https://prod-files-secure.s3.us-west-2.amazonaws.com/961c8a41-8e92-4022-aaca-e1e87c240949/b666f8d9-31ef-46c9-a214-41f962b3ba40/image.png)

<**실전! RAG 고급 기법 - Retriever (1),** 모두의AI, https://www.youtube.com/watch?v=J2AsmUODBak>

- **풍부한 정보 제공**: RAG 모델은 검색을 통해 얻은 외부 데이터를 활용하여, 보다 구체적이고 풍부한 정보를 제공할 수 있습니다.
- **실시간 정보 반영**: 최신 데이터를 검색하여 반영함으로써, 모델이 실시간으로 변화하는 정보에 대응할 수 있습니다.
- **환각 방지**: 검색을 통해 실제 데이터에 기반한 답변을 생성함으로써, 환각 현상이 발생할 위험을 줄이고 정확도를 높일 수 있습니다.

**문서 읽어들이고 요약**

[**랭체인(langchain) + PDF 문서요약, Map-Reduce (7)**](https://teddylee777.github.io/langchain/langchain-tutorial-07/)

https://teddylee777.github.io/langchain/langchain-tutorial-07/

https://gongu.copyright.or.kr/gongu/wrt/wrt/view.do?wrtSn=9002094&menuNo=200030

[운수_좋은_날.zip](https://prod-files-secure.s3.us-west-2.amazonaws.com/961c8a41-8e92-4022-aaca-e1e87c240949/8b211fe3-68e4-40bc-9d61-1f90684e5568/%EC%9A%B4%EC%88%98_%EC%A2%8B%EC%9D%80_%EB%82%A0.zip)

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 여러 개의 url 지정 가능
url1 = "https://blog.langchain.dev/week-of-1-22-24-langchain-release-notes/"
url2 = "https://blog.langchain.dev/week-of-2-5-24-langchain-release-notes/"

loader = WebBaseLoader(
    web_paths=(url1, url2),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("article-header", "article-content")
        )
    ),
)
docs = loader.load()
print(len(docs)

텍스트 파일 로딩

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('history.txt')
data = loader.load()

print(type(data))
print(len(data))
print(data)
print(len(data[0].page_content)) #첫 번째 Document 객체의 문자열의 개수
print(data[0].metadata)

디렉토리 로딩

In [ ]:
import os
from glob import glob

files = glob(os.path.join('./', '*.txt'))
files

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(path='./', glob='*.txt')

data = loader.load()

print(len(pages)

print(data[0])

**csv 파일 로딩**

한국주택금융공사_주택금융관련_지수_20160101

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='20160101.csv', encoding='cp949')
data = loader.load()

print(len(data)

print(data[0])

PDF 로딩

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_filepath = '000660_SK_2023.pdf'
loader = PyPDFLoader(pdf_filepath)
pages = loader.load()

print(len(pages))
print(pages[3])

In [ ]:
$ pip install unstructured

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader

pdf_filepath = '000660_SK_2023.pdf'

# 전체 텍스트를 단일 문서 객체로 변환
loader = UnstructuredPDFLoader(pdf_filepath)
pages = loader.load()

print(pages)
print(len(pages))
print(pages[0].page_content[:1000])

`unstructured` 라이브러리는 PDF 파일 내의 다양한 텍스트 조각(chunk)를 서로 다른 "elements"로 생성하고, 별도 설정을 하지 않으면 기본적으로 이러한 elements를 결합하여 하나의 텍스트 데이터로 반환

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

pdf_filepath = '000660_SK_2023.pdf'

loader = PyMuPDFLoader(pdf_filepath)
pages = loader.load()

print(len(pages))
print(pages[0].page_content)
print(pages[0].metadata)

langchain_community.document_loaders 모듈의 PyMuPDFLoader 클래스는 PyMuPDF를 사용하여 PDF 파일의 페이지를 로드하고, 각 페이지를 개별 document 객체로 추출합니다. 특히 PDF 문서의 자세한 메타데이터를 추출하는 데 강점이 있음


In [ ]:
from langchain_community.document_loaders import OnlinePDFLoader

# Transformers 논문을 로드
loader = OnlinePDFLoader("https://arxiv.org/pdf/1706.03762.pdf")
pages = loader.load()

print(len(pages))
print(pages[0].page_content[:1000])

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader('./')
data = loader.load()
print(len(data))
print(data[1])

https://teddylee777.github.io/langchain/rag-tutorial/

https://wikidocs.net/231364

## 종합 예제: PDF 문서의 로드를 도와주는 PDF Loader 를 활용

In [ ]:
pip install python-dotenv pypdf transformers
pip install -U langchain-community #필요할 수 있음.

실습 X, 입출력 설명으로 대체

`PyPDFLoader` 사용하여 PDF를 로드하고, 각 문서가 페이지 내용과 페이지 번호를 포함한 메타데이터를 포함하는 문서의 배열생성

**알려진 방법(최근 라이브러리로는 작동 어려움)**

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# ========== ① 문서로드 ========== #

# PDF 파일 로드
loader = PyPDFLoader("data/황순원-소나기.pdf")
document = loader.load()
document[0].page_content[:200]

# ========== ② 문서분할 ========== #

# 스플리터 지정
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",  # 분할기준
    chunk_size=3000,   # 사이즈
    chunk_overlap=500, # 중첩 사이즈
)

# 분할 실행
split_docs = text_splitter.split_documents(document)
# 총 분할된 도큐먼트 수
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

# ========== ③ Map 단계 ========== #

# Map 단계에서 처리할 프롬프트 정의
# 분할된 문서에 적용할 프롬프트 내용을 기입합니다.
# 여기서 {pages} 변수에는 분할된 문서가 차례대로 대입되니다.
map_template = """다음은 문서 중 일부 내용입니다
{pages}
이 문서 목록을 기반으로 주요 내용을 요약해 주세요.
답변:"""

# Map 프롬프트 완성
map_prompt = PromptTemplate.from_template(map_template)

# Map에서 수행할 LLMChain 정의
llm = ChatOpenAI(temperature=0,
                 model_name='gpt-3.5-turbo-16k')
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# ========== ④ Reduce 단계 ========== #

# Reduce 단계에서 처리할 프롬프트 정의
reduce_template = """다음은 요약의 집합입니다:
{doc_summaries}
이것들을 바탕으로 통합된 요약을 만들어 주세요.
답변:"""

# Reduce 프롬프트 완성
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce에서 수행할 LLMChain 정의
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# 문서의 목록을 받아들여, 이를 단일 문자열로 결합하고, 이를 LLMChain에 전달합니다.
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,
    document_variable_name="doc_summaries" # Reduce 프롬프트에 대입되는 변수
)

# Map 문서를 통합하고 순차적으로 Reduce합니다.
reduce_documents_chain = ReduceDocumentsChain(
    # 호출되는 최종 체인입니다.
    combine_documents_chain=combine_documents_chain,
    # 문서가 `StuffDocumentsChain`의 컨텍스트를 초과하는 경우
    collapse_documents_chain=combine_documents_chain,
    # 문서를 그룹화할 때의 토큰 최대 개수입니다.
    token_max=4000,
)

# ========== ⑤ Map-Reduce 통합단계 ========== #

# 문서들에 체인을 매핑하여 결합하고, 그 다음 결과들을 결합합니다.
map_reduce_chain = MapReduceDocumentsChain(
    # Map 체인
    llm_chain=map_chain,
    # Reduce 체인
    reduce_documents_chain=reduce_documents_chain,
    # 문서를 넣을 llm_chain의 변수 이름(map_template 에 정의된 변수명)
    document_variable_name="pages",
    # 출력에서 매핑 단계의 결과를 반환합니다.
    return_intermediate_steps=False,
)

# ========== ⑥ 실행 결과 ========== #

# Map-Reduce 체인 실행
# 입력: 분할된 도큐먼트(②의 결과물)
result = map_reduce_chain.run(split_docs)
# 요약결과 출력
print(result)
